In [1]:
from neupy import environment
environment.reproducible()

In [2]:
import neupy
neupy.__version__



'0.5.1'

In [3]:
import numpy
numpy.__version__



'1.12.0'

In [4]:
import platform
platform.python_version()

'3.5.2'

In [5]:
def str2bin(text, max_length=30):
    if len(text) > max_length:
        raise ValueError("Text can't contains more "
                         "than {} symbols".format(max_length))

    text = text.rjust(max_length)

    bits_list = []
    for symbol in text:
        bits = bin(ord(symbol))
        # Cut `0b` from the beggining and fill with zeros if they
        # are missed
        bits = bits[2:].zfill(8)
        bits_list.extend(map(int, bits))

    return list(bits_list)

In [6]:
a = bin(ord('t'))[2:].zfill(8)

In [ ]:
b = map(int, a)

In [ ]:
bits_list = []

In [ ]:
bits_list.extend(b)

In [ ]:
bits_list == list(map(int, a))

In [7]:
def chunker(sequence, size):
    for position in range(0, len(sequence), size):
        yield sequence[position:position + size]

def bin2str(array):
    characters = []
    for binary_symbol_code in chunker(array, size=8):
        binary_symbol_str = ''.join(map(str, binary_symbol_code))
        character = chr(int(binary_symbol_str, base=2))
        characters.append(character)
    return ''.join(characters).lstrip()

In [8]:
import numpy as np
from neupy import algorithms

def save_password(real_password, noise_level=5):
    if noise_level < 1:
        raise ValueError("`noise_level` must be equal or greater than 1.")

    binary_password = str2bin(real_password)
    bin_password_len = len(binary_password)

    data = [binary_password]

    for _ in range(noise_level):
        # The farther from the 0.5 value the less likely
        # password recovery
        noise = np.random.binomial(1, 0.55, bin_password_len)
        data.append(noise)

    dhnet = algorithms.DiscreteHopfieldNetwork(mode='sync')
    dhnet.train(np.array(data))

    return dhnet

In [9]:
import string
import random

def hamming_distance(left, right):
    left, right = np.array(left), np.array(right)
    if left.shape != right.shape:
        raise ValueError("Shapes must be equal")
    return (left != right).sum()

def generate_password(min_length=5, max_length=30):
    symbols = list(
        string.ascii_letters +
        string.digits +
        string.punctuation
    )
    password_len = random.randrange(min_length, max_length + 1)
    password = [np.random.choice(symbols) for _ in range(password_len)]
    return ''.join(password)

In [10]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [11]:
def recover_password(dhnet, broken_password):
    test = np.array(str2bin(broken_password))
    recovered_password = dhnet.predict(test)

    if recovered_password.ndim == 2:
        recovered_password = recovered_password[0, :]

    return bin2str(recovered_password)

In [12]:
my_password = "$My%Super^Secret*^&Passwd"
dhnet = save_password(my_password, noise_level=12)
recover_password(dhnet, "-My-Super-Secret---Passwd")

'$My%Super^Secret*^&Passwd'

In [13]:
_ == my_password


True

In [14]:
recover_password(dhnet, "-My-Super")

'`¢0 1\x04L{\x05C5par\x1e[ecòg}\n^&Pcñq}\x04'

In [15]:
recover_password(dhnet, "Invalid")

'`\xa05 q\x04H{\x02A5p!R>PeãòE}\n\\$\x10cqq}\x00'

In [16]:
recover_password(dhnet, "MySuperSecretPasswd")

'$M{!Super^Searet*^&Passwd'

In [17]:
import pprint
from operator import itemgetter
from collections import OrderedDict

def cutword(word, k, fromleft=False):
    if fromleft:
        return (word[-k:] if k != 0 else '').rjust(len(word))
    return (word[:k] if k != 0 else '').ljust(len(word))

n_times = 10000
cases = OrderedDict([
    ('exclude-one', (lambda x: x - 1)),
    ('exclude-quarter', (lambda x: 3 * x // 4)),
    ('exclude-half', (lambda x: x // 2)),
    ('just-one-symbol', (lambda x: 1)),
    ('empty-string', (lambda x: 0)),
])
results = OrderedDict.fromkeys(cases.keys(), 0)

for _ in range(n_times):
    real_password = generate_password(min_length=25, max_length=25)

    for casename, func in cases.items():
        n_letters = func(len(real_password))
        broken_password = cutword(real_password, k=n_letters,
                                  fromleft=True)

        dhnet = save_password(real_password, noise_level=11)
        recovered_password = recover_password(dhnet, broken_password)

        if recovered_password != real_password:
            results[casename] += 1

print("Number of fails for each test case:")
pprint.pprint(results)

Number of fails for each test case:
OrderedDict([('exclude-one', 13),
             ('exclude-quarter', 721),
             ('exclude-half', 5811),
             ('just-one-symbol', 9997),
             ('empty-string', 10000)])
